# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('consulting').getOrCreate()

In [5]:
ships_data = spark.read.csv('cruise_ship_info.csv',inferSchema = True,
                            header = True)

In [6]:
ships_data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [10]:
ships_data.head(2)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)]

In [11]:
#string index the cruiseline feature
from pyspark.ml.feature import StringIndexer

In [13]:
indexer = StringIndexer(inputCol= 'Cruise_line',outputCol = 'cruiseIndexed')

In [17]:
indexed = indexer.fit(ships_data).transform(ships_data)

In [19]:
indexed.head()

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruiseIndexed=16.0)

In [20]:
#vectorize the neended features
from pyspark.ml.feature import VectorAssembler

In [21]:
indexed.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'cruiseIndexed']

In [23]:
assembler = VectorAssembler(inputCols = ['Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density','cruiseIndexed'],outputCol = 'features')

In [24]:
df_transformed = assembler.transform(indexed)

In [25]:
df_transformed.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'cruiseIndexed',
 'features']

In [26]:
df_transformed.head()

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruiseIndexed=16.0, features=DenseVector([6.0, 30.277, 6.94, 5.94, 3.55, 42.64, 16.0]))

In [27]:
#to be used for the model
df_final = df_transformed.select(['features','crew'])

In [28]:
#split into training and testing
train_data,test_data = df_final.randomSplit([0.7,0.3])

In [29]:
#create the model
from pyspark.ml.regression import LinearRegression

In [30]:
lr = LinearRegression(labelCol='crew')

In [31]:
#train the model
lr_model = lr.fit(train_data)

In [33]:
#evaluate the model
test_results= lr_model.evaluate(test_data)

In [36]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
| -0.2983511610299594|
| -1.2692526814350984|
|   0.264661338733049|
|0.008693246653315612|
| -1.6720017459787915|
| -0.7308526290834738|
|  0.4861643307991059|
|  0.3860889836957657|
| -0.3628394027858608|
| -0.6492893844166261|
|  0.7169701747322588|
|  0.9336932727219249|
| -0.2593892909023321|
|  1.8044810465133199|
|-0.06179860536237314|
|  0.1963762499301378|
|-0.49623509599178917|
| -1.2881317716539336|
| 0.17592764045588183|
| -0.2411429464803998|
+--------------------+
only showing top 20 rows



In [37]:
test_results.r2

0.955833564984198

In [38]:
test_results.rootMeanSquaredError

0.7283630050434561